# Reggrid hydro estimator into 1 km, every 3-hr

This notebook is to reggrid hydro estimator data in into 1-km spatial resolution every 3-hr. 

In [29]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import itertools
import sys

In [30]:
# !gunzip ../example_data/Hydro_Estimator/*.gz

In [31]:
hour = 3
datetime = 201908300
datetime_comb = []
filenames = []

In [32]:
if 201908300%100 != 0:
    datetime_comb[0:hour] = list(range(datetime - 1*(hour-1), datetime + 1, 1))
else:
    datetime_24 = datetime - 76
    datetime_comb[0:(hour-1)] = list(range(datetime_24 - 1*(hour-1), datetime_24, 1))
    datetime_comb.append(datetime)
    

In [33]:
for i in range(0,hour):
    filenames.append('../example_data/Hydro_Estimator/world1hr.' + str(datetime_comb[i]))

### Extract lat, lon data for ASCII

In [34]:
%%time
latlon = np.loadtxt('../example_data/Hydro_Estimator/zworld.latlon2', delimiter="\t", dtype=str)

CPU times: user 2min 48s, sys: 6.04 s, total: 2min 54s
Wall time: 2min 53s


In [35]:
%%time
latlon_values = np.array(' '.join(latlon).split(' '))
latlon_values = list(filter(None, latlon_values))

CPU times: user 1min 10s, sys: 15.7 s, total: 1min 25s
Wall time: 1min 25s


In [36]:
lat = list(latlon_values[0:-1:2])
lon = list(latlon_values[1::2])

In [37]:
lat = np.array(lat, dtype=np.float64)
lon = np.array(lon, dtype=np.float64)

### Extract values from Hydro Estimator ASCII and convert to Rainfall rate (mm/s)

In [41]:
input_file = []

In [42]:
%%time
for i in range(0,hour):
    input_file.append([np.loadtxt(filenames[i], delimiter='\t', skiprows=1)])
    
# input_file_3 = np.loadtxt(filename_3, delimiter='\t', skiprows=1)
# input_file_2 = np.loadtxt(filename_2, delimiter='\t', skiprows=1)
# input_file_1 = np.loadtxt(filename_1, delimiter='\t', skiprows=1)

CPU times: user 9min 49s, sys: 3.81 s, total: 9min 52s
Wall time: 9min 52s


In [43]:
# check if latlon has the same cells as the values
len(lat) == len(lon) == len(input_file[1])

False

Calculating the Rainfall with equation, R = (value-2) * 0.3048. ref: https://www.star.nesdis.noaa.gov/smcd/emb/ff/digGlobalData.php  
Since GLDAS is every 3-hour, we want the rainfall rate at the 00, 03, 06, etc. that is counting the 

In [45]:
rain = []
for i in range(0,hour):
    rain.append([(np.asanyarray(input_file[i])-2)*0.3048])

RAINRATE_hydro = [sum(x)/10800 for x in zip(*rain)]   #Conver into mm/s)

### Create NetCDF file

In [47]:
# NC file setup
Hydro_estimator = Dataset('../example_data/Hydro_Estimator_netcdf/Hydro_Estimator' + str(datetime) +'.nc','w', format='NETCDF4') 

In [48]:
Hydro_estimator.description = 'Hydro Estimator RAINRATE'
## create the lat and lon dimensions.
Hydro_estimator.createDimension('latitude',len(lat))
Hydro_estimator.createDimension('longitude',len(lon))
## Define the coordinate variables. They will hold the coordinate information
lats = Hydro_estimator.createVariable('latitude','float64',('latitude',))
lons = Hydro_estimator.createVariable('longitude','float64',('longitude',))
## Assign units attributes to coordinate var data. This attaches a text attribute to each of the coordinate variables, containing the units.
lats.units = 'degrees_north'
lons.units = 'degrees_east'
## write data to coordinate vars.
lats[:] = lat
lons[:] = lon
## create the RAINRATE variable
RAINRATE = Hydro_estimator.createVariable('RAINRATE','float64',('latitude','longitude'))
## set the units attribute.
RAINRATE.units =  'mm/s'
## write data to variables.
RAINRATE[:] = RAINRATE_hydro

ValueError: cannot reshape array of size 24891111 into shape (24891111,24891111)

In [49]:
Hydro_estimator.close()